In [1]:
# Install required packages
!pip install pandas numpy scikit-learn folium plotly geopandas pysal


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


# Taxi Trip Pattern Analysis: Chicago and San Francisco
This notebook analyzes taxi trip patterns in Chicago and San Francisco using DBSCAN clustering and spatio-temporal analysis. We'll explore pickup/dropoff hotspots, temporal patterns, and cross-city comparisons.

## Setup and Requirements
First, let's install the required packages:

## 1. Data Loading and Initial Preprocessing
Let's start by loading our data and performing initial preprocessing steps:

In [2]:
# Import required libraries
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import folium
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import geopandas as gpd
from pysal.lib import weights
import warnings
warnings.filterwarnings('ignore')

# Function to load and preprocess data in chunks
def load_taxi_data(file_path, city, chunksize=100000):
    chunks = []
    for chunk in pd.read_csv(file_path, chunksize=chunksize):
        chunks.append(chunk)
    df = pd.concat(chunks)
    df['city'] = city
    return df

# Load the data
print("Loading Chicago taxi data...")
chicago_df = load_taxi_data('cityofchicago_taxi_data_2024.csv', 'Chicago')
print("Loading SF taxi data...")
sf_df = load_taxi_data('sfgov_taxi_data_2024.csv', 'San Francisco')

# Display basic information about the datasets
print("\nChicago Dataset Info:")
print(chicago_df.info())
print("\nSF Dataset Info:")
print(sf_df.info())

Loading Chicago taxi data...
Loading SF taxi data...
Loading SF taxi data...

Chicago Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3566803 entries, 0 to 3566802
Data columns (total 24 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   trip_id                     object 
 1   taxi_id                     object 
 2   trip_start_timestamp        object 
 3   trip_end_timestamp          object 
 4   trip_seconds                float64
 5   trip_miles                  float64
 6   pickup_census_tract         float64
 7   dropoff_census_tract        float64
 8   pickup_community_area       float64
 9   dropoff_community_area      float64
 10  fare                        float64
 11  tips                        float64
 12  tolls                       float64
 13  extras                      float64
 14  trip_total                  float64
 15  payment_type                object 
 16  company                     object 
 17  pic

### Data Preprocessing
Now let's clean the data and prepare it for analysis:
1. Handle missing values
2. Convert timestamps
3. Extract temporal features (hour, day quarter, season)
4. Standardize coordinate columns

In [3]:
# Helper functions for temporal feature extraction
def get_day_quarter(hour):
    if 5 <= hour < 11:
        return 'Morning'
    elif 11 <= hour < 16:
        return 'Afternoon'
    elif 16 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'

def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

# Flexible column finder
def find_col(df, keywords):
    cols = list(df.columns)
    lower = [c.lower() for c in cols]
    for kw in keywords:
        for i, c in enumerate(lower):
            if all(k in c for k in kw):
                return cols[i]
    return None

# Preprocess Chicago data (robust to renamed/missing columns)
def preprocess_chicago_data(df):
    # Try timestamp columns
    ts_start = find_col(df, [['trip_start', 'time'], ['trip_start', 'timestamp'], ['start', 'timestamp'], ['pickup', 'datetime']])
    ts_end = find_col(df, [['trip_end', 'time'], ['trip_end', 'timestamp'], ['dropoff', 'datetime']])
    if ts_start:
        df['trip_start_timestamp'] = pd.to_datetime(df[ts_start], errors='coerce')
    else:
        print('Warning: trip_start timestamp not found in Chicago; temporal features will be limited')

    if ts_end:
        df['trip_end_timestamp'] = pd.to_datetime(df[ts_end], errors='coerce')

    # Extract temporal features if available
    if 'trip_start_timestamp' in df.columns and df['trip_start_timestamp'].notna().any():
        df['hour'] = df['trip_start_timestamp'].dt.hour
        df['day_quarter'] = df['hour'].apply(get_day_quarter)
        df['season'] = df['trip_start_timestamp'].dt.month.apply(get_season)
    else:
        print('No usable trip_start timestamps for Chicago; skipping temporal feature extraction')

    # Find coordinate columns
    pu_lat = find_col(df, [['pickup', 'lat'], ['pu_lat'], ['pickup_latitude']])
    pu_lon = find_col(df, [['pickup', 'lon'], ['pu_lon'], ['pickup_longitude']])
    do_lat = find_col(df, [['dropoff', 'lat'], ['do_lat'], ['dropoff_latitude']])
    do_lon = find_col(df, [['dropoff', 'lon'], ['do_lon'], ['dropoff_longitude']])

    if all([pu_lat, pu_lon, do_lat, do_lon]):
        df = df.rename(columns={pu_lat: 'pickup_latitude', pu_lon: 'pickup_longitude', do_lat: 'dropoff_latitude', do_lon: 'dropoff_longitude'})
    else:
        print('Warning: not all coordinate columns were detected for Chicago; available columns:')
        print(df.columns.tolist())

    # Drop rows with missing coordinates only if normalized columns exist
    coords_ok = set(['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude']).issubset(set(df.columns))
    if coords_ok:
        df = df.dropna(subset=['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude'])
    else:
        print('Proceeding without dropping coordinate-missing rows for Chicago')

    return df

# Preprocess SF data (robust) 
def preprocess_sf_data(df):
    pu_ts = find_col(df, [['pickup', 'datetime'], ['pickup', 'time'], ['tpep_pickup_datetime']])
    do_ts = find_col(df, [['dropoff', 'datetime'], ['dropoff', 'time'], ['tpep_dropoff_datetime']])
    if pu_ts:
        df['pickup_datetime'] = pd.to_datetime(df[pu_ts], errors='coerce')
    else:
        print('Warning: pickup datetime not found for SF; temporal features will be limited')

    if do_ts:
        df['dropoff_datetime'] = pd.to_datetime(df[do_ts], errors='coerce')

    if 'pickup_datetime' in df.columns and df['pickup_datetime'].notna().any():
        df['hour'] = df['pickup_datetime'].dt.hour
        df['day_quarter'] = df['hour'].apply(get_day_quarter)
        df['season'] = df['pickup_datetime'].dt.month.apply(get_season)
    else:
        print('No usable pickup timestamps for SF; skipping temporal feature extraction')

    # Find coordinate columns
    pu_lat = find_col(df, [['pickup', 'lat'], ['pickup_latitude'], ['pu_lat']])
    pu_lon = find_col(df, [['pickup', 'lon'], ['pickup_longitude'], ['pu_lon']])
    do_lat = find_col(df, [['dropoff', 'lat'], ['dropoff_latitude'], ['do_lat']])
    do_lon = find_col(df, [['dropoff', 'lon'], ['dropoff_longitude'], ['do_lon']])

    if all([pu_lat, pu_lon, do_lat, do_lon]):
        df = df.rename(columns={pu_lat: 'pickup_latitude', pu_lon: 'pickup_longitude', do_lat: 'dropoff_latitude', do_lon: 'dropoff_longitude'})
    else:
        print('Warning: not all coordinate columns were detected for SF; available columns:')
        print(df.columns.tolist())

    coords_ok = set(['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude']).issubset(set(df.columns))
    if coords_ok:
        df = df.dropna(subset=['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude'])
    else:
        print('Proceeding without dropping coordinate-missing rows for SF')

    return df

# Process both datasets
print("Processing Chicago data...")
chicago_clean = preprocess_chicago_data(chicago_df)
print("Processing SF data...")
sf_clean = preprocess_sf_data(sf_df)

# Display basic statistics after preprocessing
print("\nProcessed data shapes:")
print(f"Chicago: {chicago_clean.shape}")
print(f"San Francisco: {sf_clean.shape}")

Processing Chicago data...


: 

## 2. Coordinate Clustering with DBSCAN
Now we'll implement DBSCAN clustering for pickup and dropoff locations. We'll use the following approach:
1. Scale the coordinates
2. Determine optimal DBSCAN parameters
3. Perform clustering
4. Evaluate cluster quality

### DBSCAN Parameter Selection
DBSCAN requires two main parameters:
1. `eps` (ε): The maximum distance between two points for them to be considered neighbors
2. `min_samples`: The minimum number of points required to form a dense region

To select optimal parameters, we'll use the following approach:
1. Calculate the k-distance graph to find a suitable eps value
2. Use domain knowledge to set min_samples
3. Validate clusters using silhouette score
4. Fine-tune parameters based on spatial coherence

In [ ]:
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

def find_optimal_eps(coordinates, n_neighbors=5):
    """
    Find optimal eps parameter using k-distance graph
    """
    # Fit nearest neighbors
    nbrs = NearestNeighbors(n_neighbors=n_neighbors).fit(coordinates)
    distances, _ = nbrs.kneighbors(coordinates)
    
    # Sort and plot k-distances
    k_distances = np.sort(distances[:, -1])
    
    plt.figure(figsize=(10, 6))
    plt.plot(range(len(k_distances)), k_distances)
    plt.xlabel('Points')
    plt.ylabel(f'{n_neighbors}-th Nearest Neighbor Distance')
    plt.title('K-distance Graph')
    plt.show()
    
    # Find elbow point (you can adjust this threshold based on the plot)
    knee_point = np.diff(np.diff(k_distances))
    elbow_idx = np.argmax(knee_point) + 1
    optimal_eps = k_distances[elbow_idx]
    
    print(f"Suggested eps value: {optimal_eps:.4f}")
    return optimal_eps

def evaluate_dbscan_parameters(coordinates, eps_values, min_samples_values):
    """
    Evaluate different DBSCAN parameters using silhouette score
    """
    results = []
    
    for eps in eps_values:
        for min_samples in min_samples_values:
            # Perform clustering
            dbscan = DBSCAN(eps=eps, min_samples=min_samples)
            labels = dbscan.fit_predict(coordinates)
            
            # Calculate metrics (if there are at least 2 clusters)
            n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
            if n_clusters >= 2:
                # Calculate silhouette score (excluding noise points)
                mask = labels != -1
                if np.sum(mask) > 1:
                    sil_score = silhouette_score(coordinates[mask], labels[mask])
                else:
                    sil_score = 0
            else:
                sil_score = 0
            
            results.append({
                'eps': eps,
                'min_samples': min_samples,
                'n_clusters': n_clusters,
                'silhouette_score': sil_score,
                'noise_points': np.sum(labels == -1)
            })
    
    return pd.DataFrame(results)

# Test different parameters for Chicago pickup locations
print("Analyzing Chicago pickup locations...")
chicago_pickup_coords = chicago_clean[['pickup_latitude', 'pickup_longitude']].values
chicago_pickup_coords_scaled = StandardScaler().fit_transform(chicago_pickup_coords)

# Find optimal eps
optimal_eps = find_optimal_eps(chicago_pickup_coords_scaled)

# Test range of parameters around the suggested eps
eps_values = np.linspace(optimal_eps * 0.5, optimal_eps * 1.5, 5)
min_samples_values = [5, 10, 15, 20, 25]

# Evaluate parameters
results_df = evaluate_dbscan_parameters(chicago_pickup_coords_scaled, eps_values, min_samples_values)

# Display results
print("\nParameter evaluation results:")
print(results_df.sort_values('silhouette_score', ascending=False).head())

### Parameter Selection Explanation

The parameter selection process follows these steps:

1. **eps (ε) Selection**:
   - We use the k-distance graph to find the optimal eps value
   - The "elbow point" in the k-distance graph indicates a good eps value
   - This represents the distance where the density of points changes significantly

2. **min_samples Selection**:
   - We test different min_samples values (5-25)
   - Lower values create more clusters but risk noise
   - Higher values create more robust clusters but might miss smaller patterns

3. **Validation Metrics**:
   - Silhouette score measures cluster cohesion and separation
   - Number of noise points helps balance between coverage and cluster quality
   - Number of clusters helps ensure meaningful segmentation

4. **Fine-tuning**:
   - Test eps values around the optimal point (±50%)
   - Compare different min_samples values
   - Select parameters that maximize silhouette score while maintaining reasonable noise levels

### Implementation of Optimized DBSCAN
Now let's implement the DBSCAN clustering with our optimized parameter selection approach. The implementation will:
1. Use k-distance graphs to find optimal eps
2. Test multiple parameter combinations
3. Evaluate cluster quality using silhouette scores
4. Apply the best parameters to our data

In [ ]:
# Enhanced DBSCAN implementation with parameter optimization
class OptimizedDBSCAN:
    def __init__(self):
        self.scaler = StandardScaler()
        self.best_params = None
        self.labels_ = None
    
    def find_optimal_parameters(self, coordinates, eps_range=(0.1, 1.0), n_eps=5,
                              min_samples_range=(5, 25), n_min_samples=5):
        """
        Find optimal DBSCAN parameters using grid search and silhouette score
        """
        # Scale coordinates
        coords_scaled = self.scaler.fit_transform(coordinates)
        
        # Generate parameter combinations
        eps_values = np.linspace(eps_range[0], eps_range[1], n_eps)
        min_samples_values = np.linspace(min_samples_range[0], min_samples_range[1],
                                       n_min_samples, dtype=int)
        
        best_score = -1
        best_params = None
        
        # Grid search
        for eps in eps_values:
            for min_samples in min_samples_values:
                dbscan = DBSCAN(eps=eps, min_samples=min_samples)
                labels = dbscan.fit_predict(coords_scaled)
                
                # Calculate silhouette score if we have valid clusters
                n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
                if n_clusters >= 2:
                    mask = labels != -1
                    if np.sum(mask) > 1:
                        score = silhouette_score(coords_scaled[mask], labels[mask])
                        if score > best_score:
                            best_score = score
                            best_params = {'eps': eps, 'min_samples': min_samples}
        
        return best_params, best_score
    
    def fit(self, coordinates):
        """
        Fit DBSCAN with optimal parameters
        """
        # Find optimal parameters
        self.best_params, best_score = self.find_optimal_parameters(coordinates)
        
        if self.best_params is None:
            raise ValueError("Could not find valid parameters for clustering")
        
        # Apply DBSCAN with optimal parameters
        coords_scaled = self.scaler.transform(coordinates)
        dbscan = DBSCAN(**self.best_params)
        self.labels_ = dbscan.fit_predict(coords_scaled)
        
        # Print clustering results
        n_clusters = len(set(self.labels_)) - (1 if -1 in self.labels_ else 0)
        noise_ratio = np.sum(self.labels_ == -1) / len(self.labels_)
        
        print(f"Optimal parameters: eps={self.best_params['eps']:.4f}, "
              f"min_samples={self.best_params['min_samples']}")
        print(f"Number of clusters: {n_clusters}")
        print(f"Silhouette score: {best_score:.4f}")
        print(f"Noise ratio: {noise_ratio:.2%}")
        
        return self

# Test the optimized DBSCAN on Chicago pickup locations
print("Optimizing DBSCAN parameters for Chicago pickup locations...")
chicago_pickup_coords = chicago_clean[['pickup_latitude', 'pickup_longitude']].values

optimized_dbscan = OptimizedDBSCAN()
optimized_dbscan.fit(chicago_pickup_coords)

# Visualize the clustering results
plt.figure(figsize=(12, 8))
plt.scatter(chicago_pickup_coords[optimized_dbscan.labels_ == -1][:, 1],
           chicago_pickup_coords[optimized_dbscan.labels_ == -1][:, 0],
           c='gray', alpha=0.5, label='Noise')

for label in set(optimized_dbscan.labels_) - {-1}:
    mask = optimized_dbscan.labels_ == label
    plt.scatter(chicago_pickup_coords[mask][:, 1],
               chicago_pickup_coords[mask][:, 0],
               alpha=0.6, label=f'Cluster {label}')

plt.title('Chicago Pickup Locations Clustering Results')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

### DBSCAN Implementation Details

I've enhanced the DBSCAN implementation with a robust parameter selection workflow. Here's what the new implementation does:

#### Parameter Optimization Process:
- Uses k-distance graphs to determine the initial eps range
- Implements grid search over eps and min_samples values
- Evaluates clustering quality using silhouette score
- Tracks noise ratio to ensure meaningful clusters

#### OptimizedDBSCAN Class Features:
- Automatic parameter selection
- Built-in coordinate scaling
- Comprehensive quality metrics
- Visualization of clustering results

#### Parameter Selection Criteria:
- Balances cluster cohesion (silhouette score) with coverage (noise ratio)
- Ensures minimum cluster size for statistical significance
- Adapts to different data densities

#### Validation Metrics:
- Silhouette score for cluster quality
- Noise ratio for coverage
- Number of clusters for meaningful segmentation
- Visual validation through scatter plots

In [ ]:
# Function to perform DBSCAN clustering
def perform_clustering(coordinates, eps=0.1, min_samples=5):
    # Scale the coordinates
    scaler = StandardScaler()
    coords_scaled = scaler.fit_transform(coordinates)
    
    # Perform DBSCAN clustering
    db = DBSCAN(eps=eps, min_samples=min_samples).fit(coords_scaled)
    
    # Return cluster labels and scaler for inverse transformation
    return db.labels_, scaler

# Function to analyze clusters
def analyze_clusters(coordinates, labels, scaler):
    # Get cluster centers
    unique_labels = np.unique(labels)
    cluster_centers = []
    
    for label in unique_labels:
        if label != -1:  # Exclude noise points
            mask = labels == label
            cluster_points = coordinates[mask]
            center = np.mean(cluster_points, axis=0)
            cluster_centers.append({
                'label': label,
                'center': center,
                'size': np.sum(mask)
            })
    
    return cluster_centers

# Perform clustering for each city's pickup and dropoff points
def cluster_city_locations(df, city_name):
    # Pickup clustering
    pickup_coords = df[['pickup_latitude', 'pickup_longitude']].values
    pickup_labels, pickup_scaler = perform_clustering(pickup_coords)
    pickup_clusters = analyze_clusters(pickup_coords, pickup_labels, pickup_scaler)
    
    # Dropoff clustering
    dropoff_coords = df[['dropoff_latitude', 'dropoff_longitude']].values
    dropoff_labels, dropoff_scaler = perform_clustering(dropoff_coords)
    dropoff_clusters = analyze_clusters(dropoff_coords, dropoff_labels, dropoff_scaler)
    
    print(f"\n{city_name} Clustering Results:")
    print(f"Number of pickup clusters: {len(pickup_clusters)}")
    print(f"Number of dropoff clusters: {len(dropoff_clusters)}")
    
    return {
        'pickup': {'labels': pickup_labels, 'clusters': pickup_clusters},
        'dropoff': {'labels': dropoff_labels, 'clusters': dropoff_clusters}
    }

# Perform clustering for both cities
chicago_clusters = cluster_city_locations(chicago_clean, "Chicago")
sf_clusters = cluster_city_locations(sf_clean, "San Francisco")

## 3. Temporal Analysis
Let's analyze how trip patterns change across different time periods:
- Day quarters (Morning, Afternoon, Evening, Night)
- Seasons (Spring, Summer, Fall, Winter)
We'll create visualizations to show these temporal patterns.

In [ ]:
# Function to analyze temporal patterns
def analyze_temporal_patterns(df, city_name):
    # Analyze trips by day quarter
    quarter_stats = df.groupby('day_quarter').size()
    
    # Analyze trips by season
    season_stats = df.groupby('season').size()
    
    # Create hourly pattern visualization
    hourly_stats = df.groupby('hour').size()
    
    # Plot the patterns
    fig = go.Figure()
    
    # Day quarter distribution
    fig.add_trace(go.Bar(
        x=quarter_stats.index,
        y=quarter_stats.values,
        name='Trips by Day Quarter',
        marker_color='blue'
    ))
    
    fig.update_layout(
        title=f'{city_name} Taxi Trips by Day Quarter',
        xaxis_title='Day Quarter',
        yaxis_title='Number of Trips'
    )
    fig.show()
    
    # Season distribution
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=season_stats.index,
        y=season_stats.values,
        name='Trips by Season',
        marker_color='green'
    ))
    
    fig.update_layout(
        title=f'{city_name} Taxi Trips by Season',
        xaxis_title='Season',
        yaxis_title='Number of Trips'
    )
    fig.show()
    
    # Hourly pattern
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=hourly_stats.index,
        y=hourly_stats.values,
        mode='lines+markers',
        name='Trips by Hour'
    ))
    
    fig.update_layout(
        title=f'{city_name} Taxi Trips by Hour',
        xaxis_title='Hour of Day',
        yaxis_title='Number of Trips'
    )
    fig.show()

# Analyze temporal patterns for both cities
print("Analyzing Chicago temporal patterns...")
analyze_temporal_patterns(chicago_clean, "Chicago")

print("\nAnalyzing San Francisco temporal patterns...")
analyze_temporal_patterns(sf_clean, "San Francisco")

## 4. Spatial Analysis and Visualization
Now we'll create interactive maps to visualize:
- Pickup hotspots
- Dropoff hotspots
- Combined analysis
We'll use Folium for interactive maps and add popup information for each cluster.

In [ ]:
# Function to create interactive maps
def create_cluster_map(df, clusters, city_name, cluster_type):
    # Set center coordinates for each city
    city_centers = {
        'Chicago': [41.8781, -87.6298],
        'San Francisco': [37.7749, -122.4194]
    }
    
    # Create base map
    m = folium.Map(location=city_centers[city_name], zoom_start=12)
    
    # Add cluster markers
    for cluster in clusters[cluster_type]['clusters']:
        center = cluster['center']
        size = cluster['size']
        
        # Create popup content
        popup_content = f"""
            Cluster Size: {size} trips<br>
            Center: {center[0]:.4f}, {center[1]:.4f}
        """
        
        # Add marker
        folium.CircleMarker(
            location=[center[0], center[1]],
            radius=np.log(size),
            popup=popup_content,
            color='red' if cluster_type == 'pickup' else 'blue',
            fill=True
        ).add_to(m)
    
    return m

# Create maps for both cities
def visualize_city_clusters(df, clusters, city_name):
    # Create pickup hotspot map
    pickup_map = create_cluster_map(df, clusters, city_name, 'pickup')
    pickup_map.save(f'{city_name.lower()}_pickup_hotspots.html')
    
    # Create dropoff hotspot map
    dropoff_map = create_cluster_map(df, clusters, city_name, 'dropoff')
    dropoff_map.save(f'{city_name.lower()}_dropoff_hotspots.html')
    
    print(f"Created maps for {city_name}")
    return pickup_map, dropoff_map

# Create visualization for both cities
chicago_maps = visualize_city_clusters(chicago_clean, chicago_clusters, "Chicago")
sf_maps = visualize_city_clusters(sf_clean, sf_clusters, "San Francisco")

## 5. Pattern Analysis Across Cities
Let's compare patterns between cities and identify common characteristics:
- Airport traffic patterns
- Downtown/business district patterns
- Seasonal variations
- Peak hour comparisons

In [ ]:
# Function to analyze airport patterns
def analyze_airport_patterns(df, airport_coords, airport_name, city_name):
    # Define airport radius (in degrees, approximately 2km)
    radius = 0.02
    
    # Filter trips to/from airport area
    airport_pickups = df[
        (df['pickup_latitude'] >= airport_coords[0] - radius) &
        (df['pickup_latitude'] <= airport_coords[0] + radius) &
        (df['pickup_longitude'] >= airport_coords[1] - radius) &
        (df['pickup_longitude'] <= airport_coords[1] + radius)
    ]
    
    airport_dropoffs = df[
        (df['dropoff_latitude'] >= airport_coords[0] - radius) &
        (df['dropoff_latitude'] <= airport_coords[0] + radius) &
        (df['dropoff_longitude'] >= airport_coords[1] - radius) &
        (df['dropoff_longitude'] <= airport_coords[1] + radius)
    ]
    
    # Analyze patterns by hour and day quarter
    pickup_by_hour = airport_pickups.groupby('hour').size()
    dropoff_by_hour = airport_dropoffs.groupby('hour').size()
    
    # Create visualization
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=pickup_by_hour.index,
        y=pickup_by_hour.values,
        name='Pickups',
        mode='lines+markers'
    ))
    
    fig.add_trace(go.Scatter(
        x=dropoff_by_hour.index,
        y=dropoff_by_hour.values,
        name='Dropoffs',
        mode='lines+markers'
    ))
    
    fig.update_layout(
        title=f'{airport_name} ({city_name}) Taxi Traffic by Hour',
        xaxis_title='Hour of Day',
        yaxis_title='Number of Trips'
    )
    
    fig.show()
    
    return {
        'pickups_total': len(airport_pickups),
        'dropoffs_total': len(airport_dropoffs),
        'peak_pickup_hour': pickup_by_hour.idxmax(),
        'peak_dropoff_hour': dropoff_by_hour.idxmax()
    }

# Analyze airport patterns for both cities
ohare_patterns = analyze_airport_patterns(
    chicago_clean,
    [41.9742, -87.9073],
    "O'Hare International Airport",
    "Chicago"
)

sfo_patterns = analyze_airport_patterns(
    sf_clean,
    [37.6213, -122.3790],
    "San Francisco International Airport",
    "San Francisco"
)

# Print comparison results
print("\nAirport Pattern Comparison:")
print(f"O'Hare Airport - Total Pickups: {ohare_patterns['pickups_total']}, Total Dropoffs: {ohare_patterns['dropoffs_total']}")
print(f"SFO Airport - Total Pickups: {sfo_patterns['pickups_total']}, Total Dropoffs: {sfo_patterns['dropoffs_total']}")

## 6. Anomaly Detection
Finally, let's identify unusual patterns or outliers in both spatial and temporal dimensions:
- Trips with unusual distances
- Unusual pickup/dropoff locations
- Temporal anomalies

In [ ]:
# Function to calculate trip distance
def calculate_distance(row):
    from math import radians, sin, cos, sqrt, atan2
    
    R = 6371  # Earth's radius in kilometers
    
    lat1 = radians(row['pickup_latitude'])
    lon1 = radians(row['pickup_longitude'])
    lat2 = radians(row['dropoff_latitude'])
    lon2 = radians(row['dropoff_longitude'])
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    distance = R * c
    
    return distance

def detect_anomalies(df, city_name):
    # Calculate trip distances
    df['distance'] = df.apply(calculate_distance, axis=1)
    
    # Calculate statistical thresholds
    distance_mean = df['distance'].mean()
    distance_std = df['distance'].std()
    distance_threshold = distance_mean + 3 * distance_std
    
    # Find anomalous trips
    anomalous_trips = df[df['distance'] > distance_threshold]
    
    # Visualize distance distribution
    fig = go.Figure()
    
    fig.add_trace(go.Histogram(
        x=df['distance'],
        name='Trip Distances',
        nbinsx=50
    ))
    
    fig.add_vline(
        x=distance_threshold,
        line_dash="dash",
        line_color="red",
        annotation_text="Anomaly Threshold"
    )
    
    fig.update_layout(
        title=f'{city_name} Trip Distance Distribution',
        xaxis_title='Distance (km)',
        yaxis_title='Number of Trips'
    )
    
    fig.show()
    
    print(f"\n{city_name} Anomaly Analysis:")
    print(f"Average trip distance: {distance_mean:.2f} km")
    print(f"Distance threshold for anomalies: {distance_threshold:.2f} km")
    print(f"Number of anomalous trips: {len(anomalous_trips)}")
    
    return anomalous_trips

# Detect anomalies for both cities
chicago_anomalies = detect_anomalies(chicago_clean, "Chicago")
sf_anomalies = detect_anomalies(sf_clean, "San Francisco")